In [1]:
from board_labeler import getFiles

filesDir = "labeledConstSize/"

paths = getFiles(filesDir)

In [2]:
import cv2
import re
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

stringLen = len(filesDir)
m = re.compile(r"([a-zA-Z]+(_[a-zA-Z]+)?)_")



X = [cv2.imread(x) for x in paths]
y = [m.findall(x)[0][0] for x in paths]

dictClasses = {
    "empty":0,
    "bishop_black":1,
    "bishop_white":2,
    "king_black":4,
    "king_white":5,
    "knight_black":6,
    "knight_white":7,
    "pawn_black":8,
    "pawn_white":9,
    "queen_black":10,
    "queen_white":11,
    "rook_black":12,
    "rook_white":13,
}
y = [[dictClasses[x]] for x in y]

# temp1 = [x.shape for x in X]
# temp = set(temp1)

# temp3 = [temp1.count(x) for x in temp]
# print(temp)
# print(temp3)

print(f"{len(X)} {len(y)}")



3224 3224


In [3]:
from sklearn.model_selection import train_test_split
import numpy as np

X = np.asanyarray(X)
y = np.asanyarray(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print(y_train[0:50])


[[ 0]
 [ 0]
 [ 2]
 [ 0]
 [ 0]
 [ 0]
 [ 8]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 8]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 9]
 [ 6]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 9]
 [ 0]
 [ 0]
 [ 8]
 [ 8]
 [ 9]
 [13]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 8]
 [ 9]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 9]
 [ 8]]


In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
    ]
)

def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = data_augmentation(inputs)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(500, activation = tf.nn.relu),
                                    tf.keras.layers.Dense(250, activation = tf.nn.relu),
                                    tf.keras.layers.Dense(100, activation = tf.nn.relu),
                                    tf.keras.layers.Dense(14, activation = tf.nn.softmax)])

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
model.fit(X_train,y_train,epochs=20)


Epoch 1/20
68/68 [==============================] - 1s 4ms/step - loss: 427.7690 - accuracy: 0.6162
Epoch 2/20
68/68 [==============================] - 0s 4ms/step - loss: 44.9883 - accuracy: 0.8315
Epoch 3/20
68/68 [==============================] - 0s 4ms/step - loss: 4.9271 - accuracy: 0.9056
Epoch 4/20
68/68 [==============================] - 0s 3ms/step - loss: 16.2825 - accuracy: 0.8551
Epoch 5/20
68/68 [==============================] - 0s 4ms/step - loss: 11.7868 - accuracy: 0.9014
Epoch 6/20
68/68 [==============================] - 0s 4ms/step - loss: 2.2706 - accuracy: 0.9519
Epoch 7/20
68/68 [==============================] - 0s 4ms/step - loss: 1.4995 - accuracy: 0.9532
Epoch 8/20
68/68 [==============================] - 0s 4ms/step - loss: 0.7370 - accuracy: 0.9727
Epoch 9/20
68/68 [==============================] - 0s 4ms/step - loss: 0.2836 - accuracy: 0.9815
Epoch 10/20
68/68 [==============================] - 0s 4ms/step - loss: 0.3735 - accuracy: 0.9810
Epoch 11/20
68

In [6]:
score = model.evaluate(X_test, y_test, verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

Test loss: 0.23837880790233612
Test accuracy: 0.9802631735801697


In [ ]:
model.save("models/firstmodel")

INFO:tensorflow:Assets written to: models/firstmodel\assets


In [45]:
from board_finder import getBoardCoords,imageResize

img = cv2.imread(paths[0])

predictionClass = model.predict(X_test)

dct = {v: k for k, v in dictClasses.items()}

cv2.imshow(dct[predictionClass[8].argmax()],X_test[8])
cv2.waitKey()
cv2.destroyAllWindows()

